In [6]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os
import matplotlib.pyplot as plt


Define the Generator

In [7]:
def build_generator(z_dim):
    model = tf.keras.Sequential()

    # Input layer
    model.add(layers.Dense(7*7*256, input_dim=z_dim))
    model.add(layers.Reshape((7, 7, 256)))

    # Convolutional layers
    model.add(layers.Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))

    model.add(layers.Conv2DTranspose(64, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))

    # Output layer
    model.add(layers.Conv2DTranspose(1, kernel_size=3, strides=1, padding='same', activation='tanh'))

    return model


Define the Discriminator

In [8]:
def build_discriminator(img_shape):
    model = tf.keras.Sequential()

    # Input layer
    model.add(layers.Conv2D(64, kernel_size=3, strides=2, padding='same', input_shape=img_shape))
    model.add(layers.LeakyReLU(alpha=0.01))

    # Convolutional layers
    model.add(layers.Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))

    # Output layer
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))

    return model


Define GAN Model

In [9]:
def build_gan(generator, discriminator):
    discriminator.trainable = False  # Prevent discriminator training during GAN training

    model = tf.keras.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

Define Loss and Optimizers

In [10]:
# Loss function for discriminator and generator
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Discriminator optimizer
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

# Generator optimizer
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)


Compile Models

In [11]:
discriminator.compile(loss='binary_crossentropy', optimizer=discriminator_optimizer, metrics=['accuracy'])
generator.compile(loss='binary_crossentropy', optimizer=generator_optimizer)


Training Loop

In [12]:
def train_gan(generator, discriminator, gan, dataset, z_dim, num_epochs, batch_size):
    for epoch in range(num_epochs):
        for batch in dataset:
            # Train discriminator on real images
            d_loss_real = discriminator.train_on_batch(batch, np.ones((batch_size, 1)))

            # Generate fake images
            z = np.random.normal(0, 1, (batch_size, z_dim))
            fake_images = generator.predict(z)

            # Train discriminator on fake images
            d_loss_fake = discriminator.train_on_batch(fake_images, np.zeros((batch_size, 1)))

            # Discriminator loss
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train generator
            z = np.random.normal(0, 1, (batch_size, z_dim))
            g_loss = gan.train_on_batch(z, np.ones((batch_size, 1)))

        # Print progress and generate sample images
        print(f"Epoch {epoch}/{num_epochs}, D Loss: {d_loss[0]}, G Loss: {g_loss}")
        if epoch % 10 == 0:
            generate_and_save_images(generator, epoch + 1)


 Generate and Save Sample Images

In [13]:
def generate_and_save_images(generator, epoch, z_dim=100, num_samples=16):
    z = np.random.normal(0, 1, (num_samples, z_dim))
    generated_images = generator.predict(z)
    generated_images = 0.5 * generated_images + 0.5  # Rescale to [0, 1]

    fig, axes = plt.subplots(4, 4, figsize=(8, 8))
    for i, ax in enumerate(axes.flatten()):
        ax.imshow(generated_images[i, :, :, 0], cmap='gray')
        ax.axis('off')

    plt.savefig(f'gan_generated_image_epoch_{epoch}.png')
    plt.show()


Train the GAN

In [ ]:
# Set hyperparameters
z_dim = 100
num_epochs = 100
batch_size = 64

# Create GAN model
generator = build_generator(z_dim)
discriminator = build_discriminator((64, 64, 1))
gan = build_gan(generator, discriminator)

# Load and preprocess your image dataset (replace 'your_dataset_folder' with your dataset path)
dataset_folder = r"C:\Users\Akshata S\Desktop\avatar genaration\final_images"
image_files = [os.path.join(dataset_folder, f) for f in os.listdir(dataset_folder) if f.endswith(".jpg")]

# Create a function to preprocess images
def preprocess_image(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=1)
    img = tf.image.resize(img, (128, 128))  # Adjust the size as needed
    img = (img - 127.5) / 127.5  # Normalize to [-1, 1]
    return img

# Create TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices(image_files)
dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.shuffle(len(image_files)).batch(batch_size)

# Train GAN
train_gan(generator, discriminator, gan, dataset, z_dim, num_epochs, batch_size)


Generate Avatars

In [15]:
def generate_avatar(generator, z_dim=100, num_samples=1):
    z = np.random.normal(0, 1, (num_samples, z_dim))
    generated_image = generator.predict(z)
    generated_image = 0.5 * generated_image + 0.5  # Rescale to [0, 1]
    return generated_image
